In [1]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
#from sodapy import Socrata
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import urllib.request
from selenium.webdriver.common.keys import Keys

In [2]:
date_2year_ago = (date.today() - timedelta(days=2*365))
date_today = date.today()
print('Las fechas a utilizar son: fecha dos anios atras: ' + str(date_2year_ago) + ' y fecha actual: ' + str(date_today))
# Cambiar esta variable si se necesita una fecha especifica
date_2year_ago = 20190000#int(date_2year_ago.strftime('%Y%m%d'))
date_today = int(date_today.strftime('%Y%m%d'))

Las fechas a utilizar son: fecha dos anios atras: 2019-04-08 y fecha actual: 2021-04-07


In [3]:
def first_df():
    # leemos desde la URL el json que contiene la tabla de informacion acerca de las tasas
    df = pd.read_json('https://www.datos.gov.co/resource/mcec-87by.json')
    # hacemos transformaciones a las fechas para tenerlas YYYYMMDD numericas
    df['vigenciadesde'] = pd.to_datetime(df['vigenciadesde']).dt.strftime('%Y%m%d').astype('int32')
    df['vigenciahasta'] = pd.to_datetime(df['vigenciahasta']).dt.strftime('%Y%m%d').astype('int32')
    # retornamos el data frame desde la fecha 2019
    return df[df['vigenciadesde'] >= date_2year_ago]

In [4]:
df  = first_df()
df

,valor,unidad,vigenciadesde,vigenciahasta
0,3639.62,COP,20210408,20210408
1,3645.14,COP,20210407,20210407
2,3645.79,COP,20210406,20210406
3,3678.62,COP,20210401,20210405
4,3736.91,COP,20210331,20210331
...,...,...,...,...
534,3128.07,COP,20190110,20190110
535,3164.75,COP,20190109,20190109
536,3208.56,COP,20190105,20190108
537,3241.20,COP,20190104,20190104


In [5]:
# Se agrupa por anio y mes. Se cuenta para saber cuantos registros hay por cada mes lo que equivale
# a la cantidad de dias por mes
tasa_spot = first_df()
periodo = (tasa_spot['vigenciadesde']/100).astype('int32')
data = periodo.to_list()
names = ['periodo']
new_df = pd.DataFrame(data, columns=names)
new_df.value_counts().sort_index()

periodo
201901     19
201902     19
201903     21
201904     19
201905     21
201906     19
201907     20
201908     21
201909     19
201910     22
201911     19
201912     20
202001     19
202002     20
202003     20
202004     20
202005     20
202006     18
202007     22
202008     19
202009     21
202010     22
202011     16
202012     21
202101     18
202102     19
202103     21
202104      4
dtype: int64

In [6]:
# Enter the information from those sections here
#socrata_domain = 'datos.gov.co'
#socrata_dataset_identifier = 'mcec-87by'

# App Tokens can be generated by creating an account at https://opendata.socrata.com/signup
# Tokens are optional (`None` can be used instead), though requests will be rate limited.
#
# If you choose to use a token, run the following command on the terminal (or add it to your .bashrc)
# $ export SODAPY_APPTOKEN=<token>
#credentials = json.load(open('credentials.json'))
#socrata_token = credentials['TOKEN']


In [7]:
#client = Socrata(socrata_domain, socrata_token)
#print("Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}".format(**client.__dict__))

In [8]:
#results = client.get(socrata_dataset_identifier)
#df = pd.DataFrame.from_dict(results)
#df_splt = df['vigenciadesde'].str.split('-')
#names = ['anio', 'mes', 'dia']
#data2 = df_splt.to_list()
#new_df2 = pd.DataFrame(data2, columns=names)
#new_df2.groupby(['anio', 'mes']).count()

In [9]:
# Segundo dataframe
# Se configuran las opciones del navegador para que empiece maximixado y sin extensiones habilitadas
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [12]:
# Funcion para eliminar duplicados de una lista
def remove_duplicate_list(lst):
    ''' Funcion para eliminar duplicados de una lista'''
    return list(dict.fromkeys(lst))

# Se crea instancia del driver que abrira el navegador web con las opciones definidas anteriormente
# Cambiar direccion a donde se encuentra el driver
driver_path = r'C:\Users\Efra\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options = options)
# Se le pasa el link a abrir en el navegador
driver.get('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Tasas%20de%20inter%C3%A9s%20y%20sector%20financiero&pagina=Otras%20tasas%20de%20inter%C3%A9s')

WebDriverWait(driver, 15)
driver.switch_to.frame(driver.find_element_by_id('frame_dashboard'))

# Se da click en el boton otras tasas de interes
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.XPATH,
                                      '/html/body/div[7]/div/div/table/tbody/tr/td[1]/div/div/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/table/tbody/tr/td/div')))\
    .click()

# Se da click en el menu desplegable Indicador bancario de referencia IBR
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'sui-treeview-item-toggle-collapsed')))
driver.find_elements_by_class_name('sui-treeview-item-toggle-collapsed')[1].click()


#print(len(driver.find_elements_by_xpath("//a[@href]")))
# Se guardan todos los links asocaidos al atributo o etiqueta href
href = driver.find_elements_by_xpath("//a[@href]")
href_list = []
# Se recorre la lista href en busca de los links que pertenezcan a los insumos de IBR
for i in href:
    try:
        contains = str(i.get_attribute('href')).replace('%20', '').upper()
        #print(contains)
        # Si el href contiene alguno de los strings, es guardado en una lista que contendra los enlaces a descargar
        if('IBR_PLAZOOVERNIGHTNOMINALPARAUNRANGODEFECHASDADOIQY' in contains 
           or 'IBR_PLAZOUNMESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOTRESMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOMESVENCIDONOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOSEISMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           and 'DOWNLOAD' in contains):
            src = i.get_attribute('href')
            href_list.append(src)
            #print('saca algo ' + src)
    except:
        #print(error)
        print('error href')

# Como la lista posiblemente guarde enlaces duplicados, se aplica la funcion para remover duplicados y asi descargar una sola vez
# cada archivo
href_list = remove_duplicate_list(href_list)
print('Cantidad de archivos a descargar: ' + str(len(href_list)))
for j in href_list:
    try:
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't') 
        driver.get(j)
        print('Archivo descargado asociado a: ' + j)
    except AssertionError as error:
        print(error)
        print('error link')
        
# Cambiar a 60 segundos para que descargue antes de cerrar
time.sleep(30)
driver.close()

Cantidad de archivos a descargar: 5
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.1.IBR_Plazo%20overnight%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.2.IBR_Plazo%20un%20mes%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.3.IBR_Plazo%20tres%20meses%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=e

In [13]:
# cambiar ruta de descarga a la que se usara ex: C:\Users\Efra\Downloads
def clean_ibr_df():
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    df_overnight = pd.read_excel(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx')
    df_overnight.columns = df_overnight.iloc[7]
    df_overnight.drop(df_overnight.index[0:8], inplace = True)
    df_overnight.reset_index(drop = True, inplace = True)
    df_overnight.rename(columns = {df_overnight.columns[0] : 'fecha', df_overnight.columns[3] : 'ibr_efectivo'}, inplace = True)
    #df_overnight.drop(df_overnight.columns.difference(['fecha','ibr_efectivo']), 1, inplace=True)
    df_overnight.dropna()
    return df_overnight.head(10)
clean_ibr_df()

7,fecha,Excepción,Plazo real (días calendario),ibr_efectivo,ibr_efectivo,BANCO DAVIVIENDA,Colpatria Red Multibanca,BANAGRARIO,BANCO OCCIDENTE,Helm Bank S.A.,...,Banco Gnb Sudameris S.A.,CITIBANK,The Royal Bank of Scotland S.A.,Scotiabank Colombia S.A,BANCOLOMBIA,ITAU CORPBANCA,Banco Santander Colombia,BANCO CORPBANCA,Banco Popular S.A.,BANCO DE BOGOTÁ
0,2021-04-07 00:00:00,N,1,"1,726","1,688","1,695",NaN,"1,680","1,665",NaN,...,"1,662",NaN,NaN,NaN,"1,660","1,695",NaN,NaN,NaN,"1,705"
1,2021-04-06 00:00:00,N,1,"1,722","1,684","1,695",NaN,"1,673","1,665",NaN,...,"1,661",NaN,NaN,NaN,"1,660","1,695",NaN,NaN,NaN,"1,705"
2,2021-04-05 00:00:00,N,1,"1,723","1,685","1,695",NaN,"1,675","1,665",NaN,...,"1,657",NaN,NaN,NaN,"1,665","1,695",NaN,NaN,NaN,"1,705"
3,2021-03-31 00:00:00,N,5,"1,720","1,682","1,695",NaN,"1,668","1,645",NaN,...,"1,656",NaN,NaN,NaN,"1,658","1,695",NaN,NaN,NaN,"1,705"
4,2021-03-30 00:00:00,N,1,"1,726","1,688","1,690",NaN,"1,660","1,699",NaN,...,"1,659",NaN,NaN,NaN,"1,660","1,685",NaN,NaN,NaN,"1,695"
5,2021-03-29 00:00:00,N,1,"1,715","1,677","1,690",NaN,"1,664","1,650",NaN,...,"1,661",NaN,NaN,NaN,"1,660","1,690",NaN,NaN,NaN,"1,700"
6,2021-03-26 00:00:00,N,3,"1,718","1,680","1,690",NaN,"1,665","1,660",NaN,...,"1,659",NaN,NaN,NaN,"1,670","1,690",NaN,NaN,NaN,"1,700"
7,2021-03-25 00:00:00,N,1,"1,718","1,680","1,690",NaN,"1,660","1,660",NaN,...,"1,644",NaN,NaN,NaN,"1,670","1,690",NaN,NaN,NaN,"1,700"
8,2021-03-24 00:00:00,N,1,"1,718","1,680","1,690",NaN,"1,655","1,645",NaN,...,"1,641",NaN,NaN,NaN,"1,670","1,690",NaN,NaN,NaN,"1,700"
9,2021-03-23 00:00:00,N,1,"1,708","1,670","1,690",NaN,"1,650","1,640",NaN,...,"1,627",NaN,NaN,NaN,"1,650","1,691",NaN,NaN,NaN,"1,700"


In [61]:
# cambiar ruta de descarga a la que se usara ex: C:\Users\Efra\Downloads
def clean_ibr_df():
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    #df_overnight = pd.read_excel(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx', header = 8)
    df_overnight = pd.read_excel(r'C:\Users\Efra\Downloads\1.4.IBR_Plazo mes vencido nominal para un rango de fechas dado IQY.xlsx', header = 8)
    df_overnight.rename(columns = {'Fecha (dd/mm/aaaa)' : 'Fecha', 'IBR' : 'IBR_efectivo', 'IBR.1' : 'IBR_nominal'}, inplace = True)
    df_overnight = df_overnight[['Fecha', 'IBR_efectivo', 'IBR_nominal']]
    #df_overnight.dropna(subset = ['IBR_efectivo', 'IBR_nominal'], thresh=2, inplace = True)
    #df_overnight['Fecha'] = pd.to_datetime(df_overnight['Fecha']).dt.strftime('%Y%m%d').astype('int32')
    return df_overnight
clean_ibr_df()
# cuarto DF tomar IBR.1 si IBR esta vacio y aplicar formula para cambiar a tasa efectiva

,Fecha,IBR_efectivo,IBR_nominal
0,2012-07-31 00:00:00,NaN,"5,015"
1,2012-07-24 00:00:00,NaN,"5,161"
2,2012-07-17 00:00:00,NaN,"5,160"
3,2012-07-10 00:00:00,NaN,"5,145"
4,2012-07-03 00:00:00,NaN,"5,098"
...,...,...,...
241,NaN,NaN,NaN
242,Significado de las abreviaciones:,NaN,NaN
243,S: Excepción N: Normal,NaN,NaN
244,NaN,NaN,NaN


In [62]:
r = clean_ibr_df()
r.dtypes

Fecha            object
IBR_efectivo    float64
IBR_nominal      object
dtype: object

In [53]:
df.dtypes

valor            float64
unidad            object
vigenciadesde      int32
vigenciahasta      int32
dtype: object